<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Download-and-Restore-Model-Weights" data-toc-modified-id="Download-and-Restore-Model-Weights-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download and Restore Model Weights</a></span></li><li><span><a href="#Split-Model" data-toc-modified-id="Split-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Split Model</a></span><ul class="toc-item"><li><span><a href="#Model-Part-1" data-toc-modified-id="Model-Part-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Model Part-1</a></span></li><li><span><a href="#Model-Part-2" data-toc-modified-id="Model-Part-2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Model Part-2</a></span></li><li><span><a href="#Model-Part-3" data-toc-modified-id="Model-Part-3-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Model Part-3</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#Load-Model-and-Test" data-toc-modified-id="Load-Model-and-Test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Model and Test</a></span></li><li><span><a href="#TF2ONNX" data-toc-modified-id="TF2ONNX-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TF2ONNX</a></span><ul class="toc-item"><li><span><a href="#Part:-Model-1" data-toc-modified-id="Part:-Model-1-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Part: Model-1</a></span></li><li><span><a href="#Part:-Model-2" data-toc-modified-id="Part:-Model-2-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Part: Model-2</a></span></li><li><span><a href="#Part:-Model-3" data-toc-modified-id="Part:-Model-3-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Part: Model-3</a></span></li></ul></li></ul></div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 95))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)



numpy 1.19.5
sklearn 0.0
pandas 1.1.5
ipywidgets 7.6.3
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
plotly 5.3.0
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite not installed
onnx 1.10.1
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [2]:
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'erlangai-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_magic
def netron(line):
    if not line or line.strip() == 'help':
        print('%netron file port [height]')
        return
    args = line.split()
    file, port, height = args[0], int(args[1]), int(args[2]) if len(args) == 3 else 600
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    try:
        netron.start(file, address=('0.0.0.0', port), browse=False)
    except:
        pass
    display_html(port, height)

In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import numpy as np
from tensorflow.python.saved_model.signature_constants import DEFAULT_SERVING_SIGNATURE_DEF_KEY
from tensorflow.python.saved_model.tag_constants import SERVING
import onnx
from onnxsim import simplify
import onnxruntime as ORT

In [4]:
_IMPORT('gitee.com/qrsforever/blog_source_codes/AI/tensorflow/misc/ResnetPeriodEstimator')

In [5]:
BATCH_SIZE = 1
IMAGE_W = 112
IMAGE_H = 112
NUM_FRAMES = 64
SIM_TEMPERATURE = 13.544

SAVED_MODEL_ROOT = '/data/nb_data/split_repnet_model'

## Download and Restore Model Weights

In [6]:
PATH_TO_CKPT = '/data/pretrained/cv/repnet/'

!test ! -d $PATH_TO_CKPT && mkdir -p $PATH_TO_CKPT && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/checkpoint && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00000-of-00002 && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00001-of-00002 && \
    wget -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.index

In [7]:
model = ResnetPeriodEstimator()
model.call = tf.function(model.call)
tf.train.Checkpoint(model=model).restore(f'{PATH_TO_CKPT}/ckpt-88').expect_partial()
test_inputs = np.random.randn(BATCH_SIZE, NUM_FRAMES, IMAGE_H, IMAGE_W, 3).astype(np.float32)
test_inputs_tensor = tf.convert_to_tensor(test_inputs)

In [8]:
test_outputs = model(test_inputs_tensor)
test_outputs[0].shape, test_outputs[1].shape, test_outputs[2].shape

(TensorShape([1, 64, 32]), TensorShape([1, 64, 1]), TensorShape([1, 64, 512]))

In [9]:
model.summary()

Model: "resnet_period_estimator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, None, None, 1024)  5209600   
_________________________________________________________________
conv3d (Conv3D)              multiple                  14156288  
_________________________________________________________________
batch_normalization (BatchNo multiple                  2048      
_________________________________________________________________
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
dense (Dense)                multiple                  1049088   
_________________________________________________________________
dense_1 (Dense)              multiple                  1049088   
_________________________________________________________________
transformer_layer (Transform multiple      

In [10]:
_DIR(model)

'__main__.ResnetPeriodEstimator: ["activity_regularizer", "add_loss", "add_metric", "add_update", "add_variable", "add_weight", "apply", "base_model", "base_model_layer_name", "build", "built", "call", "compile", "compiled_loss", "compiled_metrics", "compute_dtype", "compute_mask", "compute_output_shape", "compute_output_signature", "conv_3x3_layer", "conv_channels", "conv_kernel_size", "count_params", "distribute_strategy", "dropout_layer", "dropout_rate", "dtype", "dtype_policy", "dynamic", "evaluate", "evaluate_generator", "fc_layers", "finalize_state", "fit", "fit_generator", "from_config", "get_config", "get_input_at", "get_input_mask_at", "get_input_shape_at", "get_layer", "get_losses_for", "get_output_at", "get_output_mask_at", "get_output_shape_at", "get_updates_for", "get_weights", "history", "image_size", "inbound_nodes", "input", "input_mask", "input_names", "input_projection", "input_projection2", "input_shape", "input_spec", "inputs", "l2_reg_weight", "layers", "load_weigh

## Split Model

### Model Part-1

In [11]:
class ResnetPart1(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart1')
        self.base_model = tf.keras.models.Model(
            name='base_model', inputs=model.base_model.input,
            outputs=model.base_model.get_layer('conv4_block3_out').output)
    
    def call(self, x):
        x = tf.reshape(x, (-1, IMAGE_H, IMAGE_W, 3))
        return self.base_model(x)

In [12]:
model1 = ResnetPart1(model)
model_part1_outputs = model1(test_inputs_tensor)
tf.reshape(model_part1_outputs, (-1,))[:3], '-'*90, model_part1_outputs.shape

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.6783598 , -0.4574377 , -0.11176682], dtype=float32)>,
 '------------------------------------------------------------------------------------------',
 TensorShape([64, 7, 7, 1024]))

In [13]:
model1.summary(line_length=90)

Model: "ResnetPart1"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
base_model (Functional)                 (None, None, None, 1024)            5209600       
Total params: 5,209,600
Trainable params: 5,193,856
Non-trainable params: 15,744
__________________________________________________________________________________________


In [14]:
model1.save(filepath=f'{SAVED_MODEL_ROOT}/model1', save_format='tf')

INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model1/assets


/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



### Model Part-2

In [15]:
class ResnetPart2(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart2')
        self.temporal_conv_layer = layers.Conv3D(
            512, 3, padding='same', name='temporal_conv_layer',
            dilation_rate=(3, 1, 1), weights=model.temporal_conv_layers[0].get_weights())
        self.temporal_bn_layers = layers.BatchNormalization(
            name='temporal_bn_layers', weights=model.temporal_bn_layers[0].get_weights())                  
        
    def call(self, x):
        x = tf.reshape(x, [BATCH_SIZE, -1] + x.shape.as_list()[1:])
        x = self.temporal_conv_layer(x)
        x = self.temporal_bn_layers(x)
        x = tf.nn.relu(x)
        return tf.reduce_max(x, [2, 3])

In [16]:
model2 = ResnetPart2(model)
model_part2_outputs = model2(model_part1_outputs)
tf.reshape(model_part2_outputs, (-1,))[:3], '-'*60, tf.reshape(test_outputs[2], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.       , 1.186679 , 0.7266406], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.       , 1.186679 , 0.7266406], dtype=float32)>)

In [17]:
model2.summary(line_length=90)

Model: "ResnetPart2"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
temporal_conv_layer (Conv3D)            multiple                            14156288      
__________________________________________________________________________________________
temporal_bn_layers (BatchNormalization) multiple                            2048          
Total params: 14,158,336
Trainable params: 14,157,312
Non-trainable params: 1,024
__________________________________________________________________________________________


In [18]:
model2.save(filepath=f'{SAVED_MODEL_ROOT}/model2', save_format='tf')

INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model2/assets


### Model Part-3

In [19]:
class ResnetPart3(tf.keras.models.Model):
    def __init__(self, model):
        super().__init__(name='ResnetPart3')
        self.conv_3x3_layer = layers.Conv2D(
            32, 3, padding='same', activation=tf.nn.relu, name='conv_3x3_layer',
            weights=model.conv_3x3_layer.get_weights())
        
        self.input_projection1 = layers.Dense(
            512, activation=None, name='input_projection1',
            weights=model.input_projection.get_weights())
        self.input_projection2 = layers.Dense(
            512, activation=None, name='input_projection2',
            weights=model.input_projection2.get_weights())
        
        self.pos_encoding1 = tf.compat.v1.get_variable(name='pos_encoding',  initializer=model.pos_encoding.numpy())
        self.pos_encoding2 = tf.compat.v1.get_variable(name='pos_encoding2', initializer=model.pos_encoding2.numpy())
        
        self.transformer_layer1 = TransformerLayer(512, 4, 512, 0.0, True, name='transformer_layer1')
        self.transformer_layer1(tf.random.uniform((BATCH_SIZE, NUM_FRAMES, 512)))
        self.transformer_layer1.set_weights(model.transformer_layers[0].get_weights())
        
        self.transformer_layer2 = TransformerLayer(512, 4, 512, 0.0, True, name='transformer_layer2')
        self.transformer_layer2(tf.random.uniform((BATCH_SIZE, NUM_FRAMES, 512)))
        self.transformer_layer2.set_weights(model.transformer_layers2[0].get_weights())
        
        self.dropout_layer = layers.Dropout(0.25, name='dropout', weights=model.dropout_layer.get_weights())
        self.fc_layers = [
            layers.Dense(512, activation=tf.nn.relu, name='fc_0', weights=model.fc_layers[0].get_weights()),
            layers.Dense(512, activation=tf.nn.relu, name='fc_1', weights=model.fc_layers[1].get_weights()),
            layers.Dense(NUM_FRAMES//2, name='fc_layers_2', weights=model.fc_layers[2].get_weights()),
        ]
        
        self.within_period_fc_layers = [
            layers.Dense(512, activation=tf.nn.relu, name='within_period_fc_0', weights=model.within_period_fc_layers[0].get_weights()),
            layers.Dense(512, activation=tf.nn.relu, name='within_period_fc_1', weights=model.within_period_fc_layers[1].get_weights()),
            layers.Dense(1, name='within_period_fc_2', weights=model.within_period_fc_layers[2].get_weights()),
        ]
        
        # TODO ValueError: Dimensions must be equal, but are 1 and 512 for Add
        self.pos_encoding1 = tf.tile(self.pos_encoding1, multiples=[1, 1, 512])
        self.pos_encoding2 = tf.tile(self.pos_encoding2, multiples=[1, 1, 512])
                                            
    def call(self, x):
        x = get_sims_with_noloop(x, SIM_TEMPERATURE)
        x = self.conv_3x3_layer(x)
        x = tf.reshape(x, [BATCH_SIZE, NUM_FRAMES, -1])
        within_period_x = x
        
        x = self.input_projection1(x)
        x += self.pos_encoding1
        x = self.transformer_layer1(x)
        x = flatten_sequential_feats(x, BATCH_SIZE, NUM_FRAMES)
        for fc_layer in self.fc_layers:
            x = self.dropout_layer(x)
            x = fc_layer(x)
            
        within_period_x = self.input_projection2(within_period_x)
        within_period_x += self.pos_encoding2
        within_period_x = self.transformer_layer2(within_period_x)
        within_period_x = flatten_sequential_feats(within_period_x, BATCH_SIZE, NUM_FRAMES)
        for fc_layer in self.within_period_fc_layers:
            within_period_x = self.dropout_layer(within_period_x)
            within_period_x = fc_layer(within_period_x)
          
        return x, within_period_x

In [20]:
model3 = ResnetPart3(model)
model_part3_outputs = model3(model_part2_outputs)
tf.reshape(model_part3_outputs[0], (-1,))[:3], '-'*60, tf.reshape(test_outputs[0], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-12.723541 ,   0.7942119,  -1.8805947], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-12.723541  ,   0.79420686,  -1.8805971 ], dtype=float32)>)

In [21]:
model3.summary(line_length=90)

Model: "ResnetPart3"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
conv_3x3_layer (Conv2D)                 multiple                            320           
__________________________________________________________________________________________
input_projection1 (Dense)               multiple                            1049088       
__________________________________________________________________________________________
input_projection2 (Dense)               multiple                            1049088       
__________________________________________________________________________________________
transformer_layer1 (TransformerLayer)   multiple                            1577984       
__________________________________________________________________________________________
transformer_layer2 (TransformerLayer)   multiple                     

In [22]:
model3.save(filepath=f'{SAVED_MODEL_ROOT}/model3', save_format='tf')

INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model3/assets


INFO:tensorflow:Assets written to: /data/nb_data/split_repnet_model/model3/assets


### Test

In [23]:
outputs = model3(model2(model1(test_inputs_tensor)))
outputs[0].shape, outputs[1].shape

(TensorShape([1, 64, 32]), TensorShape([1, 64, 1]))

In [24]:
tf.reshape(outputs[0], (-1,))[:3], '-'*60, tf.reshape(test_outputs[0], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-12.723541 ,   0.7942119,  -1.8805947], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-12.723541  ,   0.79420686,  -1.8805971 ], dtype=float32)>)

In [25]:
tf.reshape(outputs[1], (-1,))[:3], '-'*60, tf.reshape(test_outputs[1], (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.9562206 , 0.98782516, 1.1635792 ], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.9562211 , 0.98782593, 1.1635787 ], dtype=float32)>)

## Load Model and Test

In [26]:
model2_loaded = tf.saved_model.load(f'{SAVED_MODEL_ROOT}/model2')
model2_graph_infer = model2_loaded.signatures['serving_default']

In [27]:
tf.reshape(model2_graph_infer(model_part1_outputs)['output_1'], (-1,))[:3], tf.reshape(model_part2_outputs, (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.       , 1.186679 , 0.7266406], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.       , 1.186679 , 0.7266406], dtype=float32)>)

In [28]:
tf.reshape(model2_loaded(model_part1_outputs), (-1,))[:3], tf.reshape(model_part2_outputs, (-1,))[:3]

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.       , 1.186679 , 0.7266406], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.       , 1.186679 , 0.7266406], dtype=float32)>)

## TF2ONNX

In [29]:
DEFAULT_SERVING_SIGNATURE_DEF_KEY, SERVING

('serving_default', 'serve')

### Part: Model-1

In [30]:
!python3 -m tf2onnx.convert --opset 13 \
    --tag {SERVING} --signature_def {DEFAULT_SERVING_SIGNATURE_DEF_KEY} \
    --saved-model {SAVED_MODEL_ROOT}/model1 --output {SAVED_MODEL_ROOT}/model1_opset13.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-14 21:01:02,339 - INFO - Signatures found in model: [serving_default].
2021-10-14 21:01:02,340 - INFO - Output names: ['output_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-14 21:01:05,237 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:703: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-14 21:01:06,401 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.9.2/0f28b7
2021-10-14 21:01:06,401 - INFO - Using opset <onnx, 13>
2021-10-14 21:01:12,119 - INFO - Computed 0 values for constant folding
2021-10-14 21:01:16,270

In [31]:
%netron {SAVED_MODEL_ROOT}/model1_opset13.onnx 8331 500

Serving '/data/nb_data/split_repnet_model/model1_opset13.onnx' at http://0.0.0.0:8331


In [32]:
onnx_model1 = onnx.load(f'{SAVED_MODEL_ROOT}/model1_opset13.onnx')
model_simp1, check = simplify(
    onnx_model1, dynamic_input_shape=False, 
    input_shapes={'input_1': [1, 64, 112, 112, 3]})
if not check:
    raise
model_simp1.graph.input, model_simp1.graph.output

([name: "input_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 3
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 1024
       }
     }
   }
 }
 ])

In [33]:
onnx.save(model_simp1, f'{SAVED_MODEL_ROOT}/model1_opset13_simp.onnx')

### Part: Model-2

In [34]:
!python3 -m tf2onnx.convert --opset 13 \
    --tag {SERVING} --signature_def {DEFAULT_SERVING_SIGNATURE_DEF_KEY} \
    --saved-model {SAVED_MODEL_ROOT}/model2 --output {SAVED_MODEL_ROOT}/model2_opset13.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-14 21:01:34,523 - INFO - Signatures found in model: [serving_default].
2021-10-14 21:01:34,524 - INFO - Output names: ['output_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-14 21:01:38,557 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:703: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-14 21:01:41,756 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.9.2/0f28b7
2021-10-14 21:01:41,756 - INFO - Using opset <onnx, 13>
2021-10-14 21:01:57,705 - INFO - Computed 2 values for constant folding
2021-10-14 21:02:06,954

In [35]:
%netron {SAVED_MODEL_ROOT}/model2_opset13.onnx 8332 500

Serving '/data/nb_data/split_repnet_model/model2_opset13.onnx' at http://0.0.0.0:8332


In [36]:
import onnx.helper as OH
onnx_model2 = onnx.load(f'{SAVED_MODEL_ROOT}/model2_opset13.onnx')
onnx_model2 = onnx.shape_inference.infer_shapes(onnx_model2, check_type=True) # for value_info
for i, node in enumerate(onnx_model2.graph.node):
    # print('%5d' % i, node.name)
    if 'FusedBatchNormV3' in node.name and 'Transpose' == node.op_type:
        print('-'*60)
        print(node)
        print('+'*60)
        attr = node.attribute.pop()
        if attr.ints[1] == 3:
            node.attribute.append(OH.make_attribute('perm', [0, 4, 1, 2, 3]))
        elif attr.ints[1] == 2:
            node.attribute.append(OH.make_attribute('perm', [0, 2, 3, 4, 1]))
        print('+'*60)
        print(node)
        print('-'*60)
onnx.save(onnx_model2, f'{SAVED_MODEL_ROOT}/model2_opset13_mod.onnx')
onnx_model2.graph.input, onnx_model2.graph.output

------------------------------------------------------------
input: "StatefulPartitionedCall/ResnetPart2/temporal_conv_layer/BiasAdd:0"
output: "StatefulPartitionedCall/ResnetPart2/temporal_bn_layers/FusedBatchNormV3__200:0"
name: "StatefulPartitionedCall/ResnetPart2/temporal_bn_layers/FusedBatchNormV3__200"
op_type: "Transpose"
attribute {
  name: "perm"
  ints: 0
  ints: 3
  ints: 1
  ints: 2
  type: INTS
}

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
input: "StatefulPartitionedCall/ResnetPart2/temporal_conv_layer/BiasAdd:0"
output: "StatefulPartitionedCall/ResnetPart2/temporal_bn_layers/FusedBatchNormV3__200:0"
name: "StatefulPartitionedCall/ResnetPart2/temporal_bn_layers/FusedBatchNormV3__200"
op_type: "Transpose"
attribute {
  name: "perm"
  ints: 0
  ints: 4
  ints: 1
  ints: 2
  ints: 3
  type: INTS
}

------------------------------------------------------------


([name: "input_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_param: "unk__231"
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 1024
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_param: "unk__232"
       }
       dim {
         dim_value: 512
       }
     }
   }
 }
 ])

In [37]:
model_simp2, check = simplify(
    onnx_model2, dynamic_input_shape=False, 
    input_shapes={'input_1': [1, 7, 7, 1024]})
if not check:
    raise
model_simp2.graph.input, model_simp2.graph.output

([name: "input_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 7
       }
       dim {
         dim_value: 1024
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_param: "unk__232"
       }
       dim {
         dim_value: 512
       }
     }
   }
 }
 ])

In [38]:
onnx.save(onnx_model2, f'{SAVED_MODEL_ROOT}/model2_opset13_mod_simp.onnx')

In [39]:
%netron {SAVED_MODEL_ROOT}/model2_opset13_mod_simp.onnx 8332 500

Stopping http://0.0.0.0:8332
Serving '/data/nb_data/split_repnet_model/model2_opset13_mod_simp.onnx' at http://0.0.0.0:8332


### Part: Model-3

In [40]:
!python3 -m tf2onnx.convert --opset 13 \
    --tag {SERVING} --signature_def {DEFAULT_SERVING_SIGNATURE_DEF_KEY} \
    --saved-model {SAVED_MODEL_ROOT}/model3 --output {SAVED_MODEL_ROOT}/model3_opset13.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-14 21:02:36,722 - INFO - Signatures found in model: [serving_default].
2021-10-14 21:02:36,729 - INFO - Output names: ['output_1', 'output_2']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-14 21:02:39,485 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:703: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-14 21:02:40,638 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.9.2/0f28b7
2021-10-14 21:02:40,638 - INFO - Using opset <onnx, 13>
2021-10-14 21:02:47,563 - INFO - Computed 0 values for constant folding
2021-10-14 

In [41]:
%netron {SAVED_MODEL_ROOT}/model3_opset13.onnx 8333 500

Serving '/data/nb_data/split_repnet_model/model3_opset13.onnx' at http://0.0.0.0:8333


In [42]:
onnx_model3 = onnx.load(f'{SAVED_MODEL_ROOT}/model3_opset13.onnx')
model_simp3, check = simplify(
    onnx_model3, dynamic_input_shape=False, 
    input_shapes={'input_1': [1, 64, 512]})
if not check:
    raise
model_simp3.graph.input, model_simp3.graph.output

([name: "input_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 512
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 32
       }
     }
   }
 }
 , name: "output_2"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 1
       }
     }
   }
 }
 ])

In [43]:
onnx.save(model_simp3, f'{SAVED_MODEL_ROOT}/model3_opset13_simp.onnx')

In [44]:
%netron {SAVED_MODEL_ROOT}/model3_opset13_simp.onnx 8334 500

Serving '/data/nb_data/split_repnet_model/model3_opset13_simp.onnx' at http://0.0.0.0:8334
